In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from torch.utils.data import DataLoader, Dataset, random_split

# 检查是否有可用的 GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# 加载数据
df = pd.read_csv('housing.csv')
X = df[['longitude', 'latitude', 'housing_median_age', 'total_rooms', 'total_bedrooms', 
        'population', 'households', 'median_income']].values
y = df['median_house_value'].values

# 转换为 PyTorch 数据集
class HousingDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

dataset = HousingDataset(X, y)

# 划分训练集和测试集
train_size = int(0.7 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# 数据加载器
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# 定义神经网络
class NeuralNetwork(nn.Module):
    def __init__(self, input_dim):
        super(NeuralNetwork, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1)  # 输出为1维
        )

    def forward(self, x):
        return self.fc(x).squeeze()  # squeeze去掉多余的维度

# 初始化网络、损失函数和优化器
input_dim = X.shape[1]
model = NeuralNetwork(input_dim).to(device)  # 将模型移到 GPU
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 训练网络
epochs = 4200
for epoch in range(epochs):
    model.train()
    train_loss = 0
    for X_batch, y_batch in train_loader:
        # 将数据移到 GPU
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        optimizer.zero_grad()
        y_pred = model(X_batch)
        loss = criterion(y_pred, y_batch)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    if epoch%100 == 0:
        print(f"Epoch {epoch+1}/{epochs}, Loss: {train_loss/len(train_loader):.4f}")

# 计算 R^2 分数
def calculate_r2(loader, model):
    model.eval()
    y_true, y_pred = [], []
    with torch.no_grad():
        for X_batch, y_batch in loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)  # 数据移到 GPU
            predictions = model(X_batch)
            y_true.append(y_batch)
            y_pred.append(predictions)
    y_true = torch.cat(y_true).cpu().numpy()  # 将数据从 GPU 移回 CPU
    y_pred = torch.cat(y_pred).cpu().numpy()
    ss_total = ((y_true - y_true.mean()) ** 2).sum()
    ss_residual = ((y_true - y_pred) ** 2).sum()
    r2 = 1 - ss_residual / ss_total
    return r2

# 打印训练集和测试集的 R^2 分数
r2_train = calculate_r2(train_loader, model)
r2_test = calculate_r2(test_loader, model)

print(f"R^2 (Train): {r2_train:.4f}")
print(f"R^2 (Test): {r2_test:.4f}")


Using device: cuda
Epoch 1/4200, Loss: 50042295333.3723
Epoch 101/4200, Loss: 7019335268.9051
Epoch 201/4200, Loss: 5091180203.9124
Epoch 301/4200, Loss: 4425443774.5985
Epoch 401/4200, Loss: 4224250856.6423
Epoch 501/4200, Loss: 4140403783.9416
Epoch 601/4200, Loss: 4056214242.1022
Epoch 701/4200, Loss: 3988752151.3577
Epoch 801/4200, Loss: 3988402248.8759
Epoch 901/4200, Loss: 3938023864.9927
Epoch 1001/4200, Loss: 3908475706.8613
Epoch 1101/4200, Loss: 3880205617.5182
Epoch 1201/4200, Loss: 3837577094.5401
Epoch 1301/4200, Loss: 3854102798.0146
Epoch 1401/4200, Loss: 3855905060.4380
Epoch 1501/4200, Loss: 3874655658.0438
Epoch 1601/4200, Loss: 3830316645.8394
Epoch 1701/4200, Loss: 3790127269.3723
Epoch 1801/4200, Loss: 3893784390.0730
Epoch 1901/4200, Loss: 3789117884.7299
Epoch 2001/4200, Loss: 3799993225.3431
Epoch 2101/4200, Loss: 3835575928.5255
Epoch 2201/4200, Loss: 3805483940.4380
Epoch 2301/4200, Loss: 3807232713.8102
Epoch 2401/4200, Loss: 3768659049.5766
Epoch 2501/4200, 

In [33]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim

# 加载数据
df = pd.read_csv('housing.csv')
X = df[['longitude', 'latitude', 'housing_median_age', 'total_rooms', 
        'total_bedrooms', 'population', 'households', 'median_income']]
y = df['median_house_value']

# 数据标准化函数
def standardize(data):
    mean = torch.mean(data, dim=0)
    std = torch.std(data, dim=0)
    return (data - mean) / std, mean, std

# 转换为 PyTorch 张量
X = torch.tensor(X.values, dtype=torch.float32)
y = torch.tensor(y.values, dtype=torch.float32).view(-1, 1)

# 标准化数据
X, X_mean, X_std = standardize(X)
y, y_mean, y_std = standardize(y)

# 划分数据集（70%训练，30%测试）
dataset_size = len(X)
train_size = int(0.7 * dataset_size)
test_size = dataset_size - train_size
indices = torch.randperm(dataset_size)

X_train = X[indices[:train_size]]
y_train = y[indices[:train_size]]
X_test = X[indices[train_size:]]
y_test = y[indices[train_size:]]

# 使用 GPU（如果可用）
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
X_train, y_train = X_train.to(device), y_train.to(device)
X_test, y_test = X_test.to(device), y_test.to(device)

# 定义改进的网络结构
class ImprovedNet(nn.Module):
    def __init__(self, input_dim):
        super(ImprovedNet, self).__init__()
        self.fc1 = nn.Linear(input_dim, 128)   # 第一层，128个神经元
        self.bn1 = nn.BatchNorm1d(128)        # 批归一化
        self.fc2 = nn.Linear(128, 64)         # 第二层，64个神经元
        self.bn2 = nn.BatchNorm1d(64)         # 批归一化
        self.fc3 = nn.Linear(64, 32)
        self.bn3 = nn.BatchNorm1d(32)
        self.fc4 = nn.Linear(32, 1)
        self.dropout = nn.Dropout(0.3) 

    def forward(self, x):
        x = torch.relu(self.bn1(self.fc1(x))) 
        x = self.dropout(x)# 第一层：全连接 + 批归一化 + 激活函数
        x = torch.relu(self.bn2(self.fc2(x)))
        x = self.dropout(x)# 第二层
        x = torch.relu(self.bn3(self.fc3(x))) 
        x = self.dropout(x)
        x = self.fc4(x)                       # 输出层
        return x

# 初始化网络和优化器
input_dim = X_train.shape[1]
net = ImprovedNet(input_dim).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

# 训练循环
epochs = 2000
for epoch in range(1, epochs + 1):
    
    net.train()
    optimizer.zero_grad()

    outputs = net(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()

    # 每100次输出一次损失
    if epoch % 100 == 0:
        print(f"Epoch {epoch}/{epochs}, Loss: {loss.item():.4f}")

# 在测试集上评估
net.eval()
with torch.no_grad():
    y_pred_train = net(X_train)
    y_pred_test = net(X_test)

# 计算R方
def calculate_r2(y_true, y_pred):
    ss_total = ((y_true - y_true.mean()) ** 2).sum()
    ss_residual = ((y_true - y_pred) ** 2).sum()
    return 1 - (ss_residual / ss_total)

r2_train = calculate_r2(y_train, y_pred_train)
r2_test = calculate_r2(y_test, y_pred_test)

print(f"Train R^2: {r2_train.item():.4f}")
print(f"Test R^2: {r2_test.item():.4f}")


Epoch 100/2000, Loss: 0.3433
Epoch 200/2000, Loss: 0.3061
Epoch 300/2000, Loss: 0.2835
Epoch 400/2000, Loss: 0.2698
Epoch 500/2000, Loss: 0.2576
Epoch 600/2000, Loss: 0.2531
Epoch 700/2000, Loss: 0.2489
Epoch 800/2000, Loss: 0.2423
Epoch 900/2000, Loss: 0.2358
Epoch 1000/2000, Loss: 0.2329
Epoch 1100/2000, Loss: 0.2217
Epoch 1200/2000, Loss: 0.2256
Epoch 1300/2000, Loss: 0.2259
Epoch 1400/2000, Loss: 0.2235
Epoch 1500/2000, Loss: 0.2113
Epoch 1600/2000, Loss: 0.2129
Epoch 1700/2000, Loss: 0.2063
Epoch 1800/2000, Loss: 0.2061
Epoch 1900/2000, Loss: 0.2082
Epoch 2000/2000, Loss: 0.2005
Train R^2: 0.8505
Test R^2: 0.8184


In [42]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim

# 加载数据
df = pd.read_csv('housing.csv')
X = df[['longitude', 'latitude', 'housing_median_age', 'total_rooms', 
        'total_bedrooms', 'population', 'households', 'median_income']]
y = df['median_house_value']

# 数据标准化函数
def standardize(data):
    mean = torch.mean(data, dim=0)
    std = torch.std(data, dim=0)
    return (data - mean) / std, mean, std

# 转换为 PyTorch 张量
X = torch.tensor(X.values, dtype=torch.float32)
y = torch.tensor(y.values, dtype=torch.float32).view(-1, 1)

# 标准化数据
X, X_mean, X_std = standardize(X)
y, y_mean, y_std = standardize(y)

# 划分数据集（70%训练，30%测试）
dataset_size = len(X)
train_size = int(0.7 * dataset_size)
test_size = dataset_size - train_size
indices = torch.randperm(dataset_size)

X_train = X[indices[:train_size]]
y_train = y[indices[:train_size]]
X_test = X[indices[train_size:]]
y_test = y[indices[train_size:]]

# 使用 GPU（如果可用）
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
X_train, y_train = X_train.to(device), y_train.to(device)
X_test, y_test = X_test.to(device), y_test.to(device)

# 定义更复杂的网络结构
class ComplexNet(nn.Module):
    def __init__(self, input_dim):
        super(ComplexNet, self).__init__()
        self.fc1 = nn.Linear(input_dim, 128)   # 第一层，128个神经元
        self.bn1 = nn.BatchNorm1d(128)        # 批归一化
        self.fc2 = nn.Linear(128, 64)         # 第二层，64个神经元
        self.bn2 = nn.BatchNorm1d(64)         # 批归一化
        self.fc3 = nn.Linear(64, 32)
        self.bn3 = nn.BatchNorm1d(32)
        self.fc4 = nn.Linear(32, 1)
        self.dropout = nn.Dropout(0.3) 

    def forward(self, x):
        x = torch.relu(self.bn1(self.fc1(x))) 
        x = self.dropout(x)# 第一层：全连接 + 批归一化 + 激活函数
        x = torch.relu(self.bn2(self.fc2(x)))
        x = self.dropout(x)# 第二层
        x = torch.relu(self.bn3(self.fc3(x))) 
        x = self.dropout(x)
        x = self.fc4(x)                       # 输出层
        return x

# 初始化网络和优化器
input_dim = X_train.shape[1]
net = ComplexNet(input_dim).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

# 动态调整学习率
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=100, gamma=0.1)

# 训练循环
epochs = 1000
for epoch in range(1, epochs + 1):
    net.train()
    optimizer.zero_grad()

    outputs = net(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()
    scheduler.step()

    # 每100次输出一次损失
    if epoch % 100 == 0:
        print(f"Epoch {epoch}/{epochs}, Loss: {loss.item():.4f}")

# 在测试集上评估
net.eval()
with torch.no_grad():
    y_pred_train = net(X_train)
    y_pred_test = net(X_test)

# 计算R方
def calculate_r2(y_true, y_pred):
    ss_total = ((y_true - y_true.mean()) ** 2).sum()
    ss_residual = ((y_true - y_pred) ** 2).sum()
    return 1 - (ss_residual / ss_total)

r2_train = calculate_r2(y_train, y_pred_train)
r2_test = calculate_r2(y_test, y_pred_test)

print(f"Train R^2: {r2_train.item():.4f}")
print(f"Test R^2: {r2_test.item():.4f}")


Epoch 100/1000, Loss: 0.3381
Epoch 200/1000, Loss: 0.3299
Epoch 300/1000, Loss: 0.3301
Epoch 400/1000, Loss: 0.3279
Epoch 500/1000, Loss: 0.3272
Epoch 600/1000, Loss: 0.3358
Epoch 700/1000, Loss: 0.3364
Epoch 800/1000, Loss: 0.3300
Epoch 900/1000, Loss: 0.3269
Epoch 1000/1000, Loss: 0.3309
Train R^2: 0.7307
Test R^2: 0.7460


In [46]:
# 定义函数：还原 MSE 到原始尺度
def restore_mse(original_std, standardized_mse):
    return standardized_mse * (original_std ** 2)

# 计算 R² 的函数
def calculate_r2(y_true, y_pred):
    ss_total = ((y_true - y_true.mean()) ** 2).sum()
    ss_residual = ((y_true - y_pred) ** 2).sum()
    return 1 - (ss_residual / ss_total)

# 在测试集上进行评估
net.eval()
with torch.no_grad():
    # 预测值
    y_pred_train = net(X_train)
    y_pred_test = net(X_test)
    
    # 损失计算
    train_loss = criterion(y_pred_train, y_train)
    test_loss = criterion(y_pred_test, y_test)
    
    # 还原 MSE 到原始尺度
    restored_train_loss = restore_mse(y_std.item(), train_loss.item())
    restored_test_loss = restore_mse(y_std.item(), test_loss.item())
    
    # 计算 R²
    r2_train = calculate_r2(y_train, y_pred_train)
    r2_test = calculate_r2(y_test, y_pred_test)
    
    # 输出结果
    print(f"Standardized Train MSE Loss: {train_loss.item():.4f}")
    print(f"Original Scale Train RMSE Loss: {restored_train_loss:.4f}")
    print(f"Train R²: {r2_train.item():.4f}")
    print()
    print(f"Standardized Test MSE Loss: {test_loss.item():.4f}")
    print(f"Original Scale Test RMSE Loss: {restored_test_loss:.4f}")
    print(f"Test R²: {r2_test.item():.4f}")


Standardized Train MSE Loss: 0.1444
Original Scale Train MSE Loss: 2000575726.0846
Train R²: 0.8538

Standardized Test MSE Loss: 0.2021
Original Scale Test MSE Loss: 2798613302.1529
Test R²: 0.8033


In [58]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

# 加载数据
df = pd.read_csv('housing.csv')
X = df[['longitude', 'latitude', 'housing_median_age', 'total_rooms', 
        'total_bedrooms', 'population', 'households', 'median_income']]
y = df['median_house_value']

# 数据标准化函数
def standardize(data):
    mean = torch.mean(data, dim=0)
    std = torch.std(data, dim=0)
    return (data - mean) / std, mean, std

# 转换为 PyTorch 张量
X = torch.tensor(X.values, dtype=torch.float32)
y = torch.tensor(y.values, dtype=torch.float32).view(-1, 1)

# 标准化数据
X, X_mean, X_std = standardize(X)
y, y_mean, y_std = standardize(y)

# 划分数据集（70%训练，30%测试）
dataset_size = len(X)
train_size = int(0.7 * dataset_size)
test_size = dataset_size - train_size
indices = torch.randperm(dataset_size)

X_train = X[indices[:train_size]]
y_train = y[indices[:train_size]]
X_test = X[indices[train_size:]]
y_test = y[indices[train_size:]]

# 使用 GPU（如果可用）
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
X_train, y_train = X_train.to(device), y_train.to(device)
X_test, y_test = X_test.to(device), y_test.to(device)

# 定义改进的网络结构
class Net(nn.Module):
    def __init__(self, input_dim):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_dim, 128)   # 第一层，128个神经元
        self.bn1 = nn.BatchNorm1d(128)        # 批归一化
        self.fc2 = nn.Linear(128, 64)         # 第二层，64个神经元
        self.bn2 = nn.BatchNorm1d(64)         # 批归一化
        self.fc3 = nn.Linear(64, 1)
        self.dropout = nn.Dropout(0.3) 

    def forward(self, x):
        x = torch.relu(self.bn1(self.fc1(x))) 
        x = self.dropout(x)  # 第一层：全连接 + 批归一化 + 激活函数
        x = torch.relu(self.bn2(self.fc2(x)))
        x = self.dropout(x)  # 第二层
        x = self.fc3(x)  # 输出层
        return x

# 初始化网络和优化器
input_dim = X_train.shape[1]
net = ImprovedNet(input_dim).to(device)
criterion = nn.MSELoss()  # 使用MSE作为损失函数
optimizer = optim.Adam(net.parameters(), lr=0.001)

# 训练循环
epochs = 2000
for epoch in range(1, epochs + 1):
    
    net.train()
    optimizer.zero_grad()

    outputs = net(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()

    # 每100次输出一次损失
    if epoch % 100 == 0:
        print(f"Epoch {epoch}/{epochs}, Loss: {loss.item():.4f}")

# 在测试集上评估
net.eval()
with torch.no_grad():
    y_pred_train = net(X_train)
    y_pred_test = net(X_test)

# 计算R方
def calculate_r2(y_true, y_pred):
    ss_total = ((y_true - y_true.mean()) ** 2).sum()
    ss_residual = ((y_true - y_pred) ** 2).sum()
    return 1 - (ss_residual / ss_total)

r2_train = calculate_r2(y_train, y_pred_train)
r2_test = calculate_r2(y_test, y_pred_test)

print(f"训练集 R^2: {r2_train.item():.4f}")
print(f"测试集 R^2: {r2_test.item():.4f}")

# 计算RMSE
def calculate_rmse(y_true, y_pred):
    mse = torch.mean((y_true - y_pred) ** 2)
    rmse = torch.sqrt(mse)
    return rmse

# 还原RMSE到原始尺度
def restore_rmse(y_std, rmse):
    # 确保 y_std 和 rmse 在相同的设备上
    y_std = y_std.to(rmse.device)
    return rmse * y_std

# 计算标准化后的RMSE
train_rmse = calculate_rmse(y_train, y_pred_train)
test_rmse = calculate_rmse(y_test, y_pred_test)

# 还原到原始尺度
restored_train_rmse = restore_rmse(y_std, train_rmse)
restored_test_rmse = restore_rmse(y_std, test_rmse)

# 输出时将Tensor转换为标量
print(f"训练集 RMSE: {restored_train_rmse.item():.4f}")
print(f"测试集 RMSE: {restored_test_rmse.item():.4f}")



Epoch 100/2000, Loss: 0.3233
Epoch 200/2000, Loss: 0.2810
Epoch 300/2000, Loss: 0.2577
Epoch 400/2000, Loss: 0.2494
Epoch 500/2000, Loss: 0.2407
Epoch 600/2000, Loss: 0.2368
Epoch 700/2000, Loss: 0.2298
Epoch 800/2000, Loss: 0.2275
Epoch 900/2000, Loss: 0.2227
Epoch 1000/2000, Loss: 0.2208
Epoch 1100/2000, Loss: 0.2150
Epoch 1200/2000, Loss: 0.2129
Epoch 1300/2000, Loss: 0.2121
Epoch 1400/2000, Loss: 0.2100
Epoch 1500/2000, Loss: 0.2090
Epoch 1600/2000, Loss: 0.2029
Epoch 1700/2000, Loss: 0.2054
Epoch 1800/2000, Loss: 0.2048
Epoch 1900/2000, Loss: 0.2041
Epoch 2000/2000, Loss: 0.1994
训练集 R^2: 0.8330
测试集 R^2: 0.8018
训练集 RMSE: 47822.3516
测试集 RMSE: 53047.9922


In [54]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

# 加载数据
df = pd.read_csv('housing.csv')
X = df[['longitude', 'latitude', 'housing_median_age', 'total_rooms', 
        'total_bedrooms', 'population', 'households', 'median_income']]
y = df['median_house_value']

# 数据标准化函数
def standardize(data):
    mean = torch.mean(data, dim=0)
    std = torch.std(data, dim=0)
    return (data - mean) / std, mean, std

# 转换为 PyTorch 张量
X = torch.tensor(X.values, dtype=torch.float32)
y = torch.tensor(y.values, dtype=torch.float32).view(-1, 1)

# 标准化数据
X, X_mean, X_std = standardize(X)
y, y_mean, y_std = standardize(y)

# 划分数据集（70%训练，30%测试）
dataset_size = len(X)
train_size = int(0.7 * dataset_size)
test_size = dataset_size - train_size
indices = torch.randperm(dataset_size)

X_train = X[indices[:train_size]]
y_train = y[indices[:train_size]]
X_test = X[indices[train_size:]]
y_test = y[indices[train_size:]]

# 使用 GPU（如果可用）
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
X_train, y_train = X_train.to(device), y_train.to(device)
X_test, y_test = X_test.to(device), y_test.to(device)

# 定义改进的网络结构
class ImprovedNet(nn.Module):
    def __init__(self, input_dim):
        super(ImprovedNet, self).__init__()
        self.fc1 = nn.Linear(input_dim, 128)   # 第一层，128个神经元
        self.bn1 = nn.BatchNorm1d(128)        # 批归一化
        self.fc2 = nn.Linear(128, 64)         # 第二层，64个神经元
        self.bn2 = nn.BatchNorm1d(64)         # 批归一化
        self.fc3 = nn.Linear(64, 1)
        self.dropout = nn.Dropout(0.3) 

    def forward(self, x):
        x = torch.relu(self.bn1(self.fc1(x))) 
        x = self.dropout(x)  # 第一层：全连接 + 批归一化 + 激活函数
        x = torch.relu(self.bn2(self.fc2(x)))
        x = self.dropout(x)  # 第二层
        x = self.fc3(x)  # 输出层
        return x

# 初始化网络和优化器
input_dim = X_train.shape[1]
net = ImprovedNet(input_dim).to(device)
criterion = nn.MSELoss()  # 使用MSE作为损失函数
optimizer = optim.Adam(net.parameters(), lr=0.001)

# 训练循环
epochs = 2000
for epoch in range(1, epochs + 1):
    
    net.train()
    optimizer.zero_grad()

    outputs = net(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()

    # 每100次输出一次损失
    if epoch % 100 == 0:
        print(f"Epoch {epoch}/{epochs}, Loss: {loss.item():.4f}")

# 在测试集上评估
net.eval()
with torch.no_grad():
    y_pred_train = net(X_train)
    y_pred_test = net(X_test)

# 计算R方
def calculate_r2(y_true, y_pred):
    ss_total = ((y_true - y_true.mean()) ** 2).sum()
    ss_residual = ((y_true - y_pred) ** 2).sum()
    return 1 - (ss_residual / ss_total)

r2_train = calculate_r2(y_train, y_pred_train)
r2_test = calculate_r2(y_test, y_pred_test)

print(f"Train R^2: {r2_train.item():.4f}")
print(f"Test R^2: {r2_test.item():.4f}")

# 计算RMSE
def calculate_rmse(y_true, y_pred):
    mse = torch.mean((y_true - y_pred) ** 2)
    rmse = torch.sqrt(mse)
    return rmse

# 还原RMSE到原始尺度
def restore_rmse(y_std, rmse):
    # 确保 y_std 和 rmse 在相同的设备上
    y_std = y_std.to(rmse.device)
    return rmse * y_std

# 计算标准化后的RMSE
train_rmse = calculate_rmse(y_train, y_pred_train)
test_rmse = calculate_rmse(y_test, y_pred_test)

# 还原到原始尺度
restored_train_rmse = restore_rmse(y_std, train_rmse)
restored_test_rmse = restore_rmse(y_std, test_rmse)

# 输出时将Tensor转换为标量
print(f"Standardized Train RMSE: {train_rmse.item():.4f}")
print(f"Standardized Test RMSE: {test_rmse.item():.4f}")
print(f"Original Scale Train RMSE: {restored_train_rmse.item():.4f}")
print(f"Original Scale Test RMSE: {restored_test_rmse.item():.4f}")



Epoch 100/2000, Loss: 0.3221
Epoch 200/2000, Loss: 0.2843
Epoch 300/2000, Loss: 0.2706
Epoch 400/2000, Loss: 0.2563
Epoch 500/2000, Loss: 0.2474
Epoch 600/2000, Loss: 0.2389
Epoch 700/2000, Loss: 0.2384
Epoch 800/2000, Loss: 0.2339
Epoch 900/2000, Loss: 0.2293
Epoch 1000/2000, Loss: 0.2274
Epoch 1100/2000, Loss: 0.2213
Epoch 1200/2000, Loss: 0.2198
Epoch 1300/2000, Loss: 0.2194
Epoch 1400/2000, Loss: 0.2200
Epoch 1500/2000, Loss: 0.2123
Epoch 1600/2000, Loss: 0.2172
Epoch 1700/2000, Loss: 0.2071
Epoch 1800/2000, Loss: 0.2063
Epoch 1900/2000, Loss: 0.2052
Epoch 2000/2000, Loss: 0.2050
Train R^2: 0.8325
Test R^2: 0.7978
Standardized Train RMSE: 0.4121
Standardized Test RMSE: 0.4423
Original Scale Train RMSE: 48498.4609
Original Scale Test RMSE: 52050.0117
